In [1]:
from textgenrnn import textgenrnn
import jamspell

from collections import Counter

In [2]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('/home/mitya/PycharmProjects/text_gen/models/ru_small.bin')

True

In [3]:
textgen = textgenrnn('/home/mitya/PycharmProjects/text_gen/models/bashe_300.hdf5')

In [4]:
# large model, need more comppower
#model = KeyedVectors.load('/home/mitya/PycharmProjects/text_gen/models/187/model.model')

In [5]:
letters = "А,Б,В,Г,Д,Е,Ё,Ж,З,И,Й,К,Л,М,Н,О,П,Р,С,Т,У,Ф,Х,Ц,Ч,Ш,Щ,Ъ,Ы,Ь,Э,Ю,Я"
letters = letters.lower()
letters = letters.split(',')

In [6]:

def hokku_str(n):
    hokku = []
    for i in range(n):
        hokku.append(textgen.generate(return_as_list=True, temperature=1.0))
    
    while([''] in hokku) : 
        hokku.remove(['']) 
    
    hokku = list(filter(None, hokku))

    hokku_str = ''.join(map(str, hokku))
    hokku_str = hokku_str.replace('[','').replace(']', '').replace('    ', '')
    hokku_str = hokku_str.split("'")
    
    return hokku_str


def generate_poems(num_poems, num_str):
    '''generate list of poems'''
    poems = [None]
    for i in range(num_poems):
        poems[i] = hokku_str(num_str)
        poems.append(poems[i])
    return poems

    
def delete_empty(letters, x): 
    for letter in letters:
        if  letter in x: 
            return True
        else: 
            return False


def filter_empty(poems):
    '''filter empty lists'''
    for i in range(len(poems)):
        poems[i] = [element for element in poems[i] if delete_empty(letters, element) == True]
    return poems

        
def correct(poems):
    for i in range(len(poems)):
        for j in range(len(poems[i])):
            poems[i][j] = corrector.FixFragment(str(poems[i][j]))
    return poems


def generate_end():
    text = generate_poems(num_poems, num_str)
    result = filter_empty(text)
    result = correct(result)
    return result


In [23]:
num_str = 10
num_poems = 1

In [8]:
text = generate_end()

In [9]:
text

[['Медленно выползает пчела…', 'Слово ска-', 'Под тим славным летом'],
 ['веры «пастуьей сумки»', 'гкий звон икад.'],
 ['а самом гребне перевала.', 'разве вы то из тех,', 'долгий день напролт'],
 ['а самом гребне перевала.', 'разве вы то из тех,', 'долгий день напролт']]

In [10]:
def list2string(s):
    str1 = " "    
    return (str1.join(s)) 


In [11]:
def count_words(text):
    b = 0
    for i in range(len(text)):
        a = len(list2string(text[i]).split())
        b += a
    return b


def count_words_cmplx(text):
    c = 0
    for i in range(len(text)):
        e = count_words(text[i])
        c += e
    return c

In [12]:
for i in range(len(text)):
    print(len(list2string(text[i]).split()))

9
6
12
12


In [13]:
count_words(text)

39

In [14]:
# generate poems up until some limit in words

def generate_novel(limit):
    text = []
    n_words = 0
    while True:
        #text[i] = generate_end()
        text.append(generate_end())
        n_words = count_words_cmplx(text)
    
        if n_words > limit:
            break
    return text


In [15]:
def generate_novel_test(n):
    text = []
    n_words = 0
    for i in range(n):
        #text[i] = generate_end()
        text.append(generate_end())
        
    return text


In [16]:
test = generate_novel_test(3)

In [33]:
len(test)

3

In [31]:
count_words(test[0]) + count_words(test[1]) + count_words(test[2])

175

In [32]:
count_words_cmplx(test)

175

In [24]:
text = generate_novel(100)

In [47]:
text_str = str(text)

In [48]:
text_str = text_str.replace(']', '***')
text_str = text_str.replace(',', ',\n')
text_str = text_str.replace('–', '')
text_str = text_str.replace('[', '')

In [52]:
text_str

"'В две тоска евельнется…',\n 'два-едва я обрел,\n',\n 'Возле дома,\n где не стало хозяйки,\n'***,\n 'В две тоска евельнется…',\n 'два-едва я обрел,\n',\n 'Возле дома,\n где не стало хозяйки,\n'******,\n 'а земле едва евелится',\n 'а легкий колос хватать…',\n 'Стань товарищем моим,\n',\n 'Майских долей пора.'***,\n 'а земле едва евелится',\n 'а легкий колос хватать…',\n 'Стань товарищем моим,\n',\n 'Майских долей пора.'******,\n 'И кажется мне,\n соловей на ветке ',\n 'Коне оставил кот'***,\n 'И кажется мне,\n соловей на ветке ',\n 'Коне оставил кот'******,\n 'Покидая родину',\n 'Колокол смолк вдалеке,\n',\n 'уна,\n заглянув в окно.',\n 'очная тишина.'***,\n 'Покидая родину',\n 'Колокол смолк вдалеке,\n',\n 'уна,\n заглянув в окно.',\n 'очная тишина.'******,\n 'Поздравить к постели моей.',\n 'Вторит ему азан.',\n 'Вижу себя на картине ',\n 'Прозрачная дымка.'***,\n 'Поздравить к постели моей.',\n 'Вторит ему азан.',\n 'Вижу себя на картине ',\n 'Прозрачная дымка.'*********"

In [55]:
with open('test.txt', 'wt') as output_file:
    output_file.write(text_str)
    output_file.close()

In [ ]:
def postprocess(text):
    text_str = str(text)
    text_str = text_str.replace(']', '***')
    text_str = text_str.replace(',', ',\n')
    text_str = text_str.replace('–', '')
    text_str = text_str.replace('[', '')
    with open('test.txt', 'wt') as output_file:
        output_file.write(text_str)
        output_file.close()

In [22]:
# TODO
# make poem-like string instead of list of lists
# if num_str = arbitrary and num_poems = 1 : place *** after every ]